### Adjacency method for different variables and different thresholds

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import geopandas as gpd
import numpy as np
import itertools
from itertools import combinations
from scipy import spatial
import pickle as pickle
import gudhi
from pylab import *
from mpl_toolkits.mplot3d import Axes3D
%matplotlib inline
import io
from tqdm import tqdm
from PIL import Image, ImageDraw, ImageChops, ImageFont
import shapely.geometry as geom
from shapely.ops import unary_union
import warnings

import invr

# Ignore FutureWarnings
warnings.simplefilter(action='ignore', category=FutureWarning)

#### Creating the test dataset

In [2]:
Washington_Arlington_Alexandria_DC_VA_MD_WV_Counties = gpd.read_file('./data/DCMetroArea/DC_Metro_counties/Washington_Arlington_Alexandria_DC_VA_MD_WV_Counties.shp')

In [3]:
Washington_Arlington_Alexandria_DC_VA_MD_WV_Counties

,OBJECTID,STATEFP,COUNTYFP,COUNTYNS,GEOID,NAME,NAMELSAD,LSAD,CLASSFP,MTFCC,...,Shape_Area,CBSAFP,METDIVFP,CSAFP,CBSA_Title,M_Status,MDIV_Title,CSA_Title,C_Status,geometry
0,734,54,037,01550025,54037,Jefferson,Jefferson County,06,H1,G4020,...,0.057252,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Outlying,"POLYGON ((-78.03541 39.27841, -78.03526 39.278..."
1,1266,24,009,01676636,24009,Calvert,Calvert County,06,H1,G4020,...,0.092340,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Outlying,"POLYGON ((-76.56813 38.40771, -76.57181 38.408..."
2,1297,24,033,01714670,24033,Prince George's,Prince George's County,06,H1,G4020,...,0.134028,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Central,"POLYGON ((-77.04202 38.74672, -77.04191 38.750..."
3,1299,24,017,01676992,24017,Charles,Charles County,06,H1,G4020,...,0.171903,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Outlying,"POLYGON ((-77.25505 38.56524, -77.25442 38.567..."
4,1310,24,021,01711211,24021,Frederick,Frederick County,06,H1,G4020,...,0.180931,47900,43524,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Silver Spring-Frederick-Rockville, MD","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Outlying,"POLYGON ((-77.62017 39.48470, -77.61951 39.485..."
5,1465,51,113,01494590,51113,Madison,Madison County,06,H1,G4020,...,0.085990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"POLYGON ((-78.43903 38.51684, -78.43899 38.516..."
6,1474,51,157,01673976,51157,Rappahannock,Rappahannock County,06,H1,G4020,...,0.071639,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Outlying,"POLYGON ((-78.26237 38.75855, -78.26209 38.758..."
7,1478,51,510,01498415,51510,Alexandria,Alexandria city,25,C7,G4020,...,0.004154,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Central,"POLYGON ((-77.14184 38.82552, -77.14177 38.825..."
8,1489,51,685,01498431,51685,Manassas Park,Manassas Park city,25,C7,G4020,...,0.000681,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Central,"POLYGON ((-77.47883 38.78259, -77.47528 38.786..."
9,1501,51,600,01789070,51600,Fairfax,Fairfax city,25,C7,G4020,...,0.001688,47900,47894,548,"Washington-Arlington-Alexandria, DC-VA-MD-WV",Metropolitan Statistical Area,"Washington-Arlington-Alexandria, DC-VA-MD-WV","Washington-Baltimore-Arlington, DC-MD-VA-WV-PA",Central,"POLYGON ((-77.33485 38.85374, -77.33480 38.853..."


In [4]:
us_svi = gpd.read_file('./data/DCMetroArea/SVI2020_US_county.gdb')

In [5]:
us_svi

,ST,STATE,ST_ABBR,STCNTY,COUNTY,FIPS,LOCATION,AREA_SQMI,E_TOTPOP,M_TOTPOP,...,MP_AIAN,EP_NHPI,MP_NHPI,EP_TWOMORE,MP_TWOMORE,EP_OTHERRACE,MP_OTHERRACE,Shape_Length,Shape_Area,geometry
0,01,Alabama,AL,01001,Autauga,01001,"Autauga County, Alabama",594.455838,55639,0,...,0.2,0.0,0.1,2.0,0.7,0.2,0.3,2.052888,0.150259,"MULTIPOLYGON (((-86.92120 32.65754, -86.92035 ..."
1,01,Alabama,AL,01003,Baldwin,01003,"Baldwin County, Alabama",1589.835294,218289,0,...,0.2,0.0,0.1,1.9,0.4,0.3,0.2,4.278488,0.409924,"MULTIPOLYGON (((-88.02858 30.22676, -88.02399 ..."
2,01,Alabama,AL,01005,Barbour,01005,"Barbour County, Alabama",885.007618,25026,0,...,0.3,0.0,0.1,1.3,0.6,0.6,0.4,2.569455,0.223268,"MULTIPOLYGON (((-85.74803 31.61918, -85.74543 ..."
3,01,Alabama,AL,01007,Bibb,01007,"Bibb County, Alabama",622.469288,22374,0,...,0.1,0.0,0.2,0.4,0.3,0.0,0.2,1.886874,0.156518,"MULTIPOLYGON (((-87.42194 33.00338, -87.31854 ..."
4,01,Alabama,AL,01009,Blount,01009,"Blount County, Alabama",644.890375,57755,0,...,0.1,0.1,0.1,1.8,0.3,0.3,0.4,2.391048,0.164413,"MULTIPOLYGON (((-86.96336 33.85822, -86.95967 ..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3138,56,Wyoming,WY,56037,Sweetwater,56037,"Sweetwater County, Wyoming",10426.975770,43352,0,...,0.4,0.0,0.1,2.2,0.8,0.2,0.3,7.660062,2.937306,"MULTIPOLYGON (((-110.05438 42.01103, -110.0543..."
3139,56,Wyoming,WY,56039,Teton,56039,"Teton County, Wyoming",3996.846846,23356,0,...,0.2,0.1,0.1,1.0,0.6,0.6,0.8,5.080816,1.224489,"MULTIPOLYGON (((-111.05361 44.66627, -110.7507..."
3140,56,Wyoming,WY,56041,Uinta,56041,"Uinta County, Wyoming",2081.719808,20374,0,...,0.3,0.0,0.2,2.2,0.4,0.0,0.2,3.158552,0.581250,"MULTIPOLYGON (((-111.04662 41.15604, -111.0465..."
3141,56,Wyoming,WY,56043,Washakie,56043,"Washakie County, Wyoming",2238.678265,7933,0,...,0.4,0.0,0.4,3.2,0.5,0.4,0.4,4.246732,0.650686,"MULTIPOLYGON (((-108.55063 44.15179, -108.5505..."


In [6]:
# filter us_svi to only include the DC metro area by matching Washington_Arlington_Alexandria_DC_VA_MD_WV_Counties GEOID column to us_svi STCNTY column
dc_svi = us_svi[us_svi['STCNTY'].isin(Washington_Arlington_Alexandria_DC_VA_MD_WV_Counties['GEOID'])]

In [7]:
#desired_columns = ["FIPS","geometry","EP_POV150","EP_UNEMP","EP_HBURD","EP_NOHSDP","EP_UNINSUR","EP_AGE65","EP_AGE17","EP_DISABL","EP_SNGPNT","EP_LIMENG","EP_MINRTY","EP_MUNIT","EP_MOBILE","EP_CROWD","EP_NOVEH","EP_GROUPQ"]
desired_columns = ['FIPS','geometry','EP_CROWD', 'EP_LIMENG', 'EP_MINRTY', 'EP_HBURD', 'EP_SNGPNT']


In [8]:
dc_filtered = dc_svi[desired_columns]

In [9]:
#reset index
dc_filtered = dc_filtered.reset_index(drop=True)

In [10]:
dc_filtered.head(3)

,FIPS,geometry,EP_CROWD,EP_LIMENG,EP_MINRTY,EP_HBURD,EP_SNGPNT
0,11001,"MULTIPOLYGON (((-77.11976 38.93434, -77.11253 ...",3.6,2.4,63.3,27.2,6.9
1,24009,"MULTIPOLYGON (((-76.70121 38.71276, -76.69914 ...",0.5,0.3,22.4,19.0,4.6
2,24017,"MULTIPOLYGON (((-77.27382 38.48356, -77.27240 ...",1.8,1.0,61.7,21.8,8.2


In [11]:
#standardize selected columns
for column in desired_columns[2:]:
    dc_filtered[column] = (dc_filtered[column] - dc_filtered[column].mean())/dc_filtered[column].std()

In [12]:
dc_filtered.head(3)

,FIPS,geometry,EP_CROWD,EP_LIMENG,EP_MINRTY,EP_HBURD,EP_SNGPNT
0,11001,"MULTIPOLYGON (((-77.11976 38.93434, -77.11253 ...",0.667234,-0.364646,1.159695,0.789539,0.749594
1,24009,"MULTIPOLYGON (((-76.70121 38.71276, -76.69914 ...",-1.571321,-1.034015,-0.878092,-0.840426,-0.572543
2,24017,"MULTIPOLYGON (((-77.27382 38.48356, -77.27240 ...",-0.632572,-0.810892,1.079977,-0.283852,1.496890


In [13]:
for column in desired_columns[2:]:
    dc_filtered.sort_values(by=column, inplace=True)

    # Adding a new column 'new_ID' with ID values starting from zero
    dc_filtered[column+'sortedID'] = range(len(dc_filtered))


In [14]:
#get the range of the data in the selected columns
for column in desired_columns[2:]:
    print(column, dc_filtered[column].min(), dc_filtered[column].max())

EP_CROWD -1.571320521689126 2.761364740321332
EP_LIMENG -1.0658892599707255 2.6634481627737396
EP_MINRTY -1.4112045339465336 2.3753914475510136
EP_HBURD -1.6752862962461568 2.7375466150809284
EP_SNGPNT -2.0096490339481785 1.5543738523443582


In [15]:
dc_filtered.head(3)

,FIPS,geometry,EP_CROWD,EP_LIMENG,EP_MINRTY,EP_HBURD,EP_SNGPNT,EP_CROWDsortedID,EP_LIMENGsortedID,EP_MINRTYsortedID,EP_HBURDsortedID,EP_SNGPNTsortedID
14,51157,"MULTIPOLYGON (((-78.33518 38.62875, -78.33129 ...",-0.776995,-1.002140,-1.411205,-0.363363,-2.009649,4,2,0,9,0
19,51600,"MULTIPOLYGON (((-77.33480 38.85383, -77.33333 ...",-0.560361,0.400346,0.227993,-0.582017,-1.434807,9,17,14,6,1
7,51043,"MULTIPOLYGON (((-78.15170 39.03667, -78.14240 ...",-1.426898,-0.779017,-1.261734,0.034190,-1.319838,1,6,2,17,2


In [25]:
dc_filtered

,FIPS,geometry,EP_CROWD,EP_LIMENG,EP_MINRTY,EP_HBURD,EP_SNGPNT,EP_CROWDsortedID,EP_LIMENGsortedID,EP_MINRTYsortedID,EP_HBURDsortedID,EP_SNGPNTsortedID
14,51157,"MULTIPOLYGON (((-78.335 38.629, -78.331 38.628...",-0.776995,-1.002140,-1.411205,-0.363363,-2.009649,4,2,0,9,0
19,51600,"MULTIPOLYGON (((-77.335 38.854, -77.333 38.856...",-0.560361,0.400346,0.227993,-0.582017,-1.434807,9,17,14,6,1
7,51043,"MULTIPOLYGON (((-78.152 39.037, -78.142 39.061...",-1.426898,-0.779017,-1.261734,0.034190,-1.319838,1,6,2,17,2
6,51013,"MULTIPOLYGON (((-77.172 38.893, -77.168 38.897...",0.595022,0.081599,-0.041055,-0.403118,-1.319838,19,15,11,8,3
24,54037,"MULTIPOLYGON (((-78.035 39.278, -78.033 39.280...",-0.632572,-0.970265,-1.137175,-0.442874,-0.974933,7,3,4,7,4
18,51510,"MULTIPOLYGON (((-77.144 38.811, -77.144 38.811...",1.100502,0.495970,0.417323,0.570885,-0.744996,21,18,16,19,5
1,24009,"MULTIPOLYGON (((-76.701 38.713, -76.699 38.716...",-1.571321,-1.034015,-0.878092,-0.840426,-0.572543,0,1,6,3,6
12,51113,"MULTIPOLYGON (((-78.453 38.476, -78.449 38.483...",0.378388,-1.065889,-1.231839,0.034190,-0.572543,17,0,3,16,7
11,51107,"MULTIPOLYGON (((-77.956 39.021, -77.948 39.022...",-0.776995,0.081599,0.242940,-1.675286,-0.457575,5,16,15,0,8
8,51047,"MULTIPOLYGON (((-78.229 38.533, -78.229 38.534...",0.089542,-0.141524,-0.449608,0.014312,-0.342606,11,13,9,15,9


In [16]:
# Convert the DataFrame to a GeoDataFrame
gdf = gpd.GeoDataFrame(dc_filtered, geometry='geometry')

# Set the CRS to a simple Cartesian coordinate system
gdf.crs = "EPSG:3395"  # This is a commonly used projected CRS

In [17]:
gdf

,FIPS,geometry,EP_CROWD,EP_LIMENG,EP_MINRTY,EP_HBURD,EP_SNGPNT,EP_CROWDsortedID,EP_LIMENGsortedID,EP_MINRTYsortedID,EP_HBURDsortedID,EP_SNGPNTsortedID
14,51157,"MULTIPOLYGON (((-78.335 38.629, -78.331 38.628...",-0.776995,-1.002140,-1.411205,-0.363363,-2.009649,4,2,0,9,0
19,51600,"MULTIPOLYGON (((-77.335 38.854, -77.333 38.856...",-0.560361,0.400346,0.227993,-0.582017,-1.434807,9,17,14,6,1
7,51043,"MULTIPOLYGON (((-78.152 39.037, -78.142 39.061...",-1.426898,-0.779017,-1.261734,0.034190,-1.319838,1,6,2,17,2
6,51013,"MULTIPOLYGON (((-77.172 38.893, -77.168 38.897...",0.595022,0.081599,-0.041055,-0.403118,-1.319838,19,15,11,8,3
24,54037,"MULTIPOLYGON (((-78.035 39.278, -78.033 39.280...",-0.632572,-0.970265,-1.137175,-0.442874,-0.974933,7,3,4,7,4
18,51510,"MULTIPOLYGON (((-77.144 38.811, -77.144 38.811...",1.100502,0.495970,0.417323,0.570885,-0.744996,21,18,16,19,5
1,24009,"MULTIPOLYGON (((-76.701 38.713, -76.699 38.716...",-1.571321,-1.034015,-0.878092,-0.840426,-0.572543,0,1,6,3,6
12,51113,"MULTIPOLYGON (((-78.453 38.476, -78.449 38.483...",0.378388,-1.065889,-1.231839,0.034190,-0.572543,17,0,3,16,7
11,51107,"MULTIPOLYGON (((-77.956 39.021, -77.948 39.022...",-0.776995,0.081599,0.242940,-1.675286,-0.457575,5,16,15,0,8
8,51047,"MULTIPOLYGON (((-78.229 38.533, -78.229 38.534...",0.089542,-0.141524,-0.449608,0.014312,-0.342606,11,13,9,15,9


Generate adjacent counties

In [18]:
def generate_adjacent_counties(dataframe,filtration_threshold,variable_name):
    filtered_df = dataframe[dataframe[variable_name] < filtration_threshold]

    # Perform a spatial join to find adjacent precincts
    adjacent_counties = gpd.sjoin(filtered_df, filtered_df, predicate='intersects', how='left')

    # Filter the results to include only the adjacent states
    adjacent_counties = adjacent_counties.query(variable_name+'sortedID_left != '+variable_name+'sortedID_right')

    # Group the resulting dataframe by the original precinct Name and create a list of adjacent precinct Name
    adjacent_counties = adjacent_counties.groupby(variable_name+'sortedID_left')[variable_name+'sortedID_right'].apply(list).reset_index()

    adjacent_counties.rename(columns={variable_name+'sortedID_left': 'county', variable_name+'sortedID_right': 'adjacent'}, inplace=True)

    adjacencies_list = adjacent_counties['adjacent'].tolist()

    merged_df = pd.merge(adjacent_counties, dataframe, left_on='county',right_on=variable_name+'sortedID', how='left')
    merged_df = gpd.GeoDataFrame(merged_df, geometry='geometry')

    return adjacencies_list,merged_df

In [19]:
def form_simplicial_complex(adjacent_county_list):
    max_dimension = 3

    V = []
    V = invr.incremental_vr(V, adjacent_county_list, max_dimension)

    return V
    

In [20]:
def fig2img(fig):
     #convert matplot fig to image and return it

     buf = io.BytesIO()
     fig.savefig(buf)
     buf.seek(0)
     img = Image.open(buf)
     return img

In [21]:
def plot_simplicial_complex(dataframe,V,variable_name):

    v_sortedID = variable_name+'sortedID'
    
    # #city centroids
    # city_coordinates = {city.v_sortedID: np.array((city.geometry.centroid.x, city.geometry.centroid.y)) for _, city in dataframe.iterrows()}
    # Construct city centroids dictionary
    city_coordinates = {
        row[v_sortedID]: np.array((row['geometry'].centroid.x, row['geometry'].centroid.y)) 
        for _, row in dataframe.iterrows()
    }

    print(city_coordinates)

    # Create a figure and axis
    fig, ax = plt.subplots(figsize=(8,8))
    ax.set_axis_off() 

    # Plot the "wyoming_svi" DataFrame
    dataframe.plot(ax=ax, edgecolor='black', linewidth=0.3, color="white")

    # # Plot the centroid of the large square with values
    # for i, row in dataframe.iterrows():
    #     centroid = row['geometry'].centroid
    #     text_to_display = f"FIPS: {row['FIPS']}"
    #     # plt.text(centroid.x, centroid.y, str(row['FIPS']), fontsize=15, ha='center', color="black")
    #     plt.text(centroid.x, centroid.y, text_to_display, fontsize=15, ha='center', color="black")

    for edge_or_traingle in V:

        
        if len(edge_or_traingle) == 2:
            # Plot an edge
            ax.plot(*zip(*[city_coordinates[vertex] for vertex in edge_or_traingle]), color='red', linewidth=2)
            # img = fig2img(fig)
            # list_gif.append(img)
        elif len(edge_or_traingle) == 3:
            # Plot a triangle
            ax.add_patch(plt.Polygon([city_coordinates[vertex] for vertex in edge_or_traingle], color='green', alpha=0.2))
            # img = fig2img(fig)
            # list_gif.append(img)
    plt.show()
    plt.close()

    #return list_gif #deleted  plot_simplicial_complex(dataframe,V,list_gif) --> list part

##### Multiple variable code starts here

In [22]:
# dc - 'EP_CROWD', 'EP_LIMENG', 'EP_MINRTY', 'EP_HBURD', 'EP_SNGPNT'
# define a dictionary wich has name and a value
selected_variables_and_threshold = {'EP_CROWD': 0.5, 'EP_LIMENG': 0, 'EP_MINRTY': 0.5, 'EP_HBURD': 0.5, 'EP_SNGPNT': 0.5}

In [23]:
# create a empty dictionary
edges_and_traingles_for_each_variable = {}

In [26]:
for variable_name, threshold in selected_variables_and_threshold.items():

    adjacencies_list,adjacent_counties_df = generate_adjacent_counties(gdf,threshold,variable_name)

    V = form_simplicial_complex(adjacencies_list)

    #add V list to the edges_and_traingles_for_each_variable dictionary with the key as the variable name
    edges_and_traingles_for_each_variable[variable_name] = V

    print(V)

    # # store the list of images for each variable
    # list_img = []

    # plot the simplicial complex
    # plot_simplicial_complex(gdf,V,variable_name)
    


[[0], [1], [2], [1, 2], [3], [4], [2, 4], [5], [1, 5], [2, 5], [1, 2, 5], [3, 5], [6], [2, 6], [7], [1, 7], [5, 7], [1, 5, 7], [8], [0, 8], [9], [10], [6, 10], [11], [2, 11], [4, 11], [2, 4, 11], [6, 11], [2, 6, 11], [10, 11], [6, 10, 11], [12], [5, 12], [8, 12], [9, 12], [13], [6, 13], [10, 13], [6, 10, 13], [14], [1, 14], [2, 14], [1, 2, 14], [4, 14], [2, 4, 14], [15], [12, 15], [16], [3, 16], [5, 16], [3, 5, 16], [12, 16], [5, 12, 16], [17], [4, 17], [11, 17], [4, 11, 17], [18], [2, 18], [5, 18], [2, 5, 18], [6, 18], [2, 6, 18], [12, 18], [5, 12, 18]]
[[0], [1], [2], [0, 2], [3], [4], [2, 4], [5], [1, 5], [6], [3, 6], [4, 6], [7], [2, 7], [4, 7], [2, 4, 7], [6, 7], [4, 6, 7], [8], [9], [7, 9], [8, 9], [10], [0, 10], [2, 10], [0, 2, 10], [7, 10], [2, 7, 10], [8, 10], [11], [8, 11], [10, 11], [8, 10, 11]]
[[0], [1], [0, 1], [2], [1, 2], [3], [0, 3], [4], [2, 4], [5], [0, 5], [1, 5], [0, 1, 5], [2, 5], [1, 2, 5], [6], [7], [8], [0, 8], [3, 8], [0, 3, 8], [5, 8], [0, 5, 8], [9], [10], [